In [289]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [290]:
df=pd.read_csv("clean_data.csv")

In [291]:
df.head()

,Unnamed: 0,location,total_sqft,bath,price,bhk
0,0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [292]:
df = df.drop("Unnamed: 0", axis=1)

In [293]:
df.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1200.0,2.0,130.0,3
4,1st Block Jayanagar,1235.0,2.0,148.0,2


In [294]:
df.shape

(7415, 5)

In [340]:
df.isnull().sum()

location      0
total_sqft    0
bath          0
price         0
bhk           0
dtype: int64

In [295]:
# encoder = OneHotEncoder(sparse_output=False)
# encoded_array = encoder.fit_transform(df[['location']])  # ✅ NumPy array output

#encoded_df = pd.get_dummies(df, columns=['location'], drop_first=False)

In [296]:
#encoded_array

In [297]:
X=df.drop(columns=["price"])
y=df["price"]

#train, test split

In [298]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [299]:
print(X_train.shape)
print(X_test.shape)

(5932, 4)
(1483, 4)


In [300]:
column_trans = make_column_transformer(
    (OneHotEncoder(sparse_output=False), ["location"]),
    remainder="passthrough")

In [301]:
print(column_trans)#ye koi value nhi karta h 

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(sparse_output=False),
                                 ['location'])])


In [302]:
column_trans

,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,None
,sparse_output,False


In [303]:

scaler=StandardScaler()

In [304]:
regression=LinearRegression()


In [305]:
pipe=make_pipeline(column_trans, scaler,regression)

In [306]:
pipe.fit(X_train, y_train)

,steps,"[('columntransformer', ...), ('standardscaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [307]:
X_test_linear=pipe.predict(X_test)

In [308]:
r2_score(y_test, X_test_linear)

0.799592999765451

In [321]:
r2 = r2_score(y_test, X_test_linear)
mse = mean_squared_error(y_test, X_test_linear)
mae = mean_absolute_error(y_test, X_test_linear)

print("R² Score:", r2)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

R² Score: 0.799592999765451
Mean Squared Error: 1418.2670698562717
Mean Absolute Error: 19.547206440825953


Jab tumne pehle Linear Regression pipeline chalaya, usne X_test ko transform karke NumPy array bana diya hoga. 

Ab jab tum Lasso pipeline chalate ho, toh woh ColumnTransformer ke through named columns expect karta hai — lekin NumPy array mein column names nahi hote.
isliye x, y ke name same nhi rkhna h jo linear regression me use hua h kyoki wha pr overwrite hota h 


In [310]:
#applying Lasso
from sklearn.linear_model import Lasso

lasso = Lasso()
pipe1 = make_pipeline(column_trans, scaler, lasso)


In [311]:
pipe1.fit(X_train, y_train)


,steps,"[('columntransformer', ...), ('standardscaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [322]:
X_test_lass = pipe1.predict(X_test)

In [323]:
r2_score(X_test_lass, y_test)

0.7852333485982914

In [324]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, X_test_lass)
mse = mean_squared_error(y_test, X_test_lass)
mae = mean_absolute_error(y_test, X_test_lass)

print("R² Score:", r2)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)


R² Score: 0.7888595216390604
Mean Squared Error: 1494.2271837937383
Mean Absolute Error: 21.003281814045085


In [325]:
#applying Ridge
from sklearn.linear_model import Ridge
model=Ridge()


In [326]:
pipe2 = make_pipeline(column_trans, scaler, model)

In [327]:
pipe2.fit(X_train, y_train)

,steps,"[('columntransformer', ...), ('standardscaler', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('onehotencoder', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
print("Input to model:\n", input)

print("Any NaN present:", input.isnull().values.any())


Input to model:
 <bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x0000019C14C3E1D0>>


In [328]:
x_test_ridge=pipe2.predict(X_test)

In [329]:
r2_score(x_test_ridge, y_test)

0.8103272438692726

In [336]:
print("linear regression ", r2_score(y_test, X_test_linear))
print("lasso",r2_score(X_test_lass, y_test))
print("Ridge",r2_score(x_test_ridge, y_test))

linear regression  0.799592999765451
lasso 0.7852333485982914
Ridge 0.8103272438692726


In [334]:
r2_score(x_test_ridge, y_test)

0.8103272438692726

In [337]:
import pickle
pickle.dump(pipe2, open("RidgeModel.pkl", 'wb'))